# Define processing scripts for GCB models


# decompose_GCB
this function decomposes each GCB model using the STL method

In [43]:
#import xarray as xr
#import decompose as stl

In [44]:
## data directory
#dir_data = '/local/data/artemis/workspace/gloege/LDEO-FFN/data/GCB_models/GCB_cut'

## model dictionary
#dict_model = {'CNRM-ESM':f'{dir_data}/CNRM-ESM2_spco2_1x1_198201-201712.nc',
#             'CCSM-BEC':f'{dir_data}/CCSM-BEC_spco2_1x1_198201-201712.nc',
#             'MPI':f'{dir_data}/MPI_spco2_1x1_198201-201712.nc',
#             'NEMO-planktom':f'{dir_data}/NEMO-planktom_spco2_1x1_198201-201712.nc',
#             'NorESM':f'{dir_data}/NorESM_spco2_1x1_198201-201712.nc',
#             'REcoM_jra':f'{dir_data}/REcoM_jra_spco2_1x1_198201-201712.nc',
#             'IPSL-PISCES':f'{dir_data}/IPSL-PISCES_spco2_1x1_198201-201512.nc'}

In [45]:
#ds = xr.open_dataset(dict_model['MPI'])

In [46]:
#ds = xr.open_dataset(dict_model['CNRM-ESM'])
### Make sure correct order
#ds = ds.transpose('time', 'lat', 'lon')
#ds
###======================================
### STL Decomposition
###======================================
### 0. Load raw data
#data = ds[f'spco2'].copy()


In [47]:
###======================================
### STL Decomposition
###======================================
### 0. Load raw data
#data = ds[f'spco2'].copy()
#data.shape

In [48]:
### 1. Detrend it 
#data_detrend = stl.detrend(data, dim='time')

In [49]:
#data_detrend.shape

In [50]:
#data.shape

In [ ]:
def decompose_GCB(model=None, fl_path=None, dir_out=None):
    '''
    decompose_pco2
    this function decomposes pCO2 into various time scales
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    fl_path = this is a path to the model netcdf (best to loop over a dictionary)
    dir_out = directory where output will be stored. 
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This assumes ERA interm is here:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/ERA_interim/
    You will need to change this if this is not correct
    
    and data is output to this directory:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/CO2_flux/
    
    '''        
    ###======================================
    ### Load dataset
    ###======================================
    ds = xr.open_dataset(fl_path)

    ### Make sure correct order
    ds = ds.transpose('time', 'lat', 'lon')
        
    ###======================================
    ### STL Decomposition
    ###======================================
    ### 0. Load raw data
    data = ds[f'spco2'].copy()
    
    ### 1. Detrend it 
    data_detrend = stl.detrend(data, dim='time')
    
    ### 2. Remove seasonal cycle
    # 2.1 -- seasonal cycle
    data_seasonal = stl.seasonal_cycle(data_detrend, dim='time', period=12)
    
    # 2.2 -- de-season the data
    data_deseason = data_detrend - data_seasonal
    
    ### 3. calculate the LOWESS
    data_lowess = stl.lowess(data_deseason, dim='time', lo_pts=12*10, lo_delta=0.01)

    ### 4. Residual term -- not explained by trend or seasonal cycle 
    data_residual = data_deseason - data_lowess
    
    ### Low frequency of residual
    data_residual_low = stl.lowess(data_residual, dim='time', lo_pts=12, lo_delta=0.01)

    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        'spco2': (['time', 'lat', 'lon'], data.transpose('time','lat','lon') ),
        'spco2_detrend': (['time', 'lat', 'lon'], data_detrend.transpose('time','lat','lon') ),
        'spco2_dec': (['time','lat', 'lon'], data_lowess.transpose('time','lat','lon') ),
        'spco2_seasonal': (['time','lat', 'lon'], data_seasonal.transpose('time','lat','lon') ),
        'spco2_residual': (['time','lat', 'lon'], data_residual.transpose('time','lat','lon') ),
        'spco2_residual_low': (['time','lat', 'lon'], data_residual_low.transpose('time','lat','lon') ),
        },

        coords={
        'time': (['time'], ds['time']),
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        })
        
    ### save file
    ds_out.to_netcdf(f'{dir_out}/{model}_spco2-decompose_1x1_198201-201712.nc') 

# Calculate pCO2 statistics

In [ ]:
def calculate_GCB_statistics(model=None, fl_mod=None, fl_som=None, dir_out=None):
    """
    calculate_pco2_statistics(model=None, member=None)
    model   = [MPI, CESM, or GFDL]
    member  = "member you want to process"
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Notes
    =========
    
    
    L. Gloege 2019
    """

        ## Load model
    #data_dir='/local/data/artemis/workspace/gloege/LDEO-FFN/data/GCB_models/GCB_decompose'
    #f'{data_dir}/CCSM-BEC_spco2-decompose_1x1_198201-201712.nc'
   
    ## Load SOM-FFN
    #dir_products = '/local/data/artemis/workspace/gloege/products'
    #fl_som=f'{dir_products}/SOM-FFN_decomposed/SOM-FFNv2016_decomposed_1982-2015.nc'
    
    ###======================================
    ### Load model
    ###====================================== 
    ## Load models
    ds_mod = xr.open_dataset(fl_mod)
    ## Load SOM-FFN --> need to roll and change longitude
    ds_tmp = xr.open_dataset(fl_som)
    ds_som = ds_tmp.roll(lon=180, roll_coords='lon')
    ds_som['lon'] = ds_mod.lon
    #ds_som = ds_som.sel(time=ds_mod.time)
    #ds_mod = ds_mod.where(ds_som['time']==ds_mod['time'])
    ds_mod = ds_mod.sel(time=ds_som['time'])
    
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {
        ## Bias
        'bias': (['lat', 'lon'], sk.avg_error(ds_mod['spco2'], 
                                              ds_som['spco2']),
                 {'units':'uatm'}), 
        'bias_detrend':(['lat','lon'], sk.avg_error(ds_mod['spco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                       {'units':'uatm'}),
        'bias_dec': (['lat', 'lon'], sk.avg_error(ds_mod['spco2_dec'], 
                                                    ds_som['spco2_dec']),
                       {'units':'uatm'}),
        'bias_seasonal': (['lat', 'lon'], sk.avg_error(ds_mod['spco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                       {'units':'uatm'}),
        'bias_residual': (['lat', 'lon'], sk.avg_error(ds_mod['spco2_residual'], 
                                                      ds_som['spco2_residual']),
                       {'units':'uatm'}),
        'bias_residual_low': (['lat', 'lon'], sk.avg_error(ds_mod['spco2_residual_low'], 
                                                      ds_som['spco2_residual_low']),
                       {'units':'uatm'}),

        ## average absolute error
        'aae': (['lat', 'lon'], sk.avg_abs_error(ds_mod['spco2'], 
                                              ds_som['spco2']),
                 {'units':'uatm'}), 
        'aae_detrend':(['lat','lon'], sk.avg_abs_error(ds_mod['spco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                       {'units':'uatm'}),
        'aae_dec': (['lat', 'lon'], sk.avg_abs_error(ds_mod['spco2_dec'], 
                                                    ds_som['spco2_dec']),
                       {'units':'uatm'}),
        'aae_seasonal': (['lat', 'lon'], sk.avg_abs_error(ds_mod['spco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                       {'units':'uatm'}),
        'aae_residual': (['lat', 'lon'], sk.avg_abs_error(ds_mod['spco2_residual'], 
                                                      ds_som['spco2_residual']),
                       {'units':'uatm'}),
        'aae_residual_low': (['lat', 'lon'], sk.avg_abs_error(ds_mod['spco2_residual_low'], 
                                                      ds_som['spco2_residual_low']),
                       {'units':'uatm'}),
            
       
        ## Amplitude ratio
        'amp-ratio': (['lat', 'lon'], sk.amp_ratio(ds_mod['spco2'], 
                                              ds_som['spco2']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_detrend':(['lat','lon'], sk.amp_ratio(ds_mod['spco2_detrend'], 
                                                    ds_som['spco2_detrend']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_dec': (['lat', 'lon'], sk.amp_ratio(ds_mod['spco2_dec'], 
                                                    ds_som['spco2_dec']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_seasonal': (['lat', 'lon'], sk.amp_ratio(ds_mod['spco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual': (['lat', 'lon'], sk.amp_ratio(ds_mod['spco2_residual'], 
                                                      ds_som['spco2_residual']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual_low': (['lat', 'lon'], sk.amp_ratio(ds_mod['spco2_residual_low'], 
                                                      ds_som['spco2_residual_low']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
            
        
        ## correlation
        'corr': (['lat', 'lon'], sk.correlation(ds_mod['spco2'], 
                                              ds_som['spco2']),
            {'units':'dimensionless'}),
        'corr_detrend':(['lat','lon'], sk.correlation(ds_mod['spco2_detrend'], 
                                                    ds_som['spco2_detrend']),
            {'units':'dimensionless'}),
        'corr_dec': (['lat', 'lon'], sk.correlation(ds_mod['spco2_dec'], 
                                                    ds_som['spco2_dec']),
            {'units':'dimensionless'}),
        'corr_seasonal': (['lat', 'lon'], sk.correlation(ds_mod['spco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
            {'units':'dimensionless'}),
        'corr_residual': (['lat', 'lon'], sk.correlation(ds_mod['spco2_residual'], 
                                                      ds_som['spco2_residual']),
            {'units':'dimensionless'}),
        'corr_residual_low': (['lat', 'lon'], sk.correlation(ds_mod['spco2_residual_low'], 
                                                      ds_som['spco2_residual_low']),
            {'units':'dimensionless'}),
            
            
        ## Normalized standard deviation
        'std-star': (['lat', 'lon'], sk.std_star(ds_mod['spco2'], 
                                              ds_som['spco2']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_detrend':(['lat','lon'], sk.std_star(ds_mod['spco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_dec': (['lat', 'lon'], sk.std_star(ds_mod['spco2_dec'], 
                                                    ds_som['spco2_dec']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_seasonal': (['lat', 'lon'], sk.std_star(ds_mod['spco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual': (['lat', 'lon'], sk.std_star(ds_mod['spco2_residual'], 
                                                      ds_som['spco2_residual']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual_low': (['lat', 'lon'], sk.std_star(ds_mod['spco2_residual_low'], 
                                                      ds_som['spco2_residual_low']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'})
        },

        coords={
        'lat': (['lat'], ds_mod['lat']),
        'lon': (['lon'], ds_mod['lon']),
        })
        
    # save file
    ds_out.to_netcdf(f'{dir_out}/stats_spco2_{model}_SOMFFN.nc') 